# Recuperação da Informação e Busca na Web - 2018.1

### Atividade: Lab 02 - Parte 2 - PageRank
### Aluno: Johanny de Lucena Santos

## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Nesta atividade você vai exercitar o algoritmo PageRank. De forma específica, você vai fazer o seguinte:

<ul>
    <ul>
        <li>Assistir o vídeo e (re)implementar o PageRank como descrito neste vídeo (https://www.youtube.com/watch?v=zv4OVNWfVt4).

<ul>
    <li>É importante que você realmente digite o código durante o vídeo (pausando quando necessário), isso é mais efetivo para o aprendizado do que somente copiar e colar do github por exemplo. Tente entender o código, ele é bem próximo ao que foi explicado em sala de aula.</li>

<li>Essa é somente uma das muitas implementações do PageRank disponíveis na Web. Você pode escolher outra se assim desejar.</li></ul>
<p>   
<li>Aplique sua implementação nos dados disponíveis em http://snap.stanford.edu/data/soc-sign-bitcoinotc.html. Esses dados representam links entre investidores de bitcoin. Cada investidor pode receber uma nota de acordo com outras pessoas que fizeram transações com ele(a). **Vamos considerar somente as pessoas que receberam notas iguais ou maior a oito. Ou seja, somente as arestas que tenham peso maior ou igual à oito**.
    <p>
<li>Uma vez calculado o PageRank, gere visualizações dos resultados usando a ferramenta Gephi (https://gephi.org/).
   <br> 
    <br>
Feitos os passos acima, responda as seguintes perguntas:
<p>
<li>Quantas iterações o PageRank precisou rodar até atingir convergência?</li>
<li>Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?</li>
<li>Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?</li>

### Source (citation)

S. Kumar, F. Spezzano, V.S. Subrahmanian, C. Faloutsos. Edge Weight Prediction in Weighted Signed Networks. IEEE International Conference on Data Mining (ICDM), 2016.

### Importações

In [74]:
import numpy as np
import pandas as pd

## Definição do dataset a ser utilizado
Aqui eu defino o conjunto de dados a ser utilizado, incluindo os nomes das colunas, bem como removendo a coluna TIME, que será desnecessária para as manipulações do conjunto de dados.

In [103]:
dataset = pd.read_csv('../data/soc-sign-bitcoinotc.csv')
dataset.columns = ['SOURCE_ID', 'TARGET_ID', 'RATING', 'TIME']
dataset = dataset.drop('TIME', axis=1)

### Prévia do dataset

In [117]:
dataset.head()

,SOURCE_ID,TARGET_ID,RATING
3,13,16,8
4,13,10,8
9,21,1,8
10,21,10,8
11,21,8,9


### Filtragem dos dados
Vamos considerar somente as pessoas que receberam notas iguais ou maior a oito. Ou seja, somente as arestas que tenham peso maior ou igual à oito (filtrar os dados para os elementos que possuem a coluna Rating maior ou igual a 8)

In [118]:
dataset    = dataset.loc[dataset['RATING'] >= 8]
source_ids = list(dataset['SOURCE_ID'])
target_ids = list(dataset['TARGET_ID'])
nodes_ids  = list(set(source_ids).union(set(target_ids)))

dataset.head()

,SOURCE_ID,TARGET_ID,RATING
3,13,16,8
4,13,10,8
9,21,1,8
10,21,10,8
11,21,8,9


## Criando grafo de interação entre investidores de Bitcoins

In [129]:
grafo = {node:[] for node in nodes_ids}

for i in range(len(source_ids)):
    src = source_ids[i]
    tgt = target_ids[i]
    grafo[src].append(tgt)

## Gerando a matriz de adjacência
Criando a matriz H inicialmente zerada, e em seguida, preenchemos com 1 todos que possuem links de saída, dividindo-o pelo número de links que saem dele, ou seja, o número de pessoas que investiram dele.

In [135]:
h = [[0 for i in range(len(nodes_ids))] for j in range(len(nodes_ids))]

for i in range(len(nodes_ids)):
    for j in range(len(nodes)):
        src = nodes_ids[i]
        tgt = nodes_ids[j]
        links_saida = len(grafo[src])
        
        if (tgt in grafo[src]):
            h[j][i] = float(1)/float(links_saida)
        else:
            h[j][i] = 0
            
h = np.matrix(h)

### Calculo do PageRank
Aqui, definimos o fator d = 0.15 e criamos a matriz B com todos os valores 1, e depois multiplicando por 1/n, onde n é o número de nós. Em seguida, calculamos o PageRank utilizando M = (1-d)*H + d*B

In [140]:
d = 0.15
fator = float(1)/float(len(nodes_ids))

#Matriz temporaria com todos os valores em 1
matriz_temp = np.matrix(
    [[1 for i in range(len(nodes_ids))] for j in range(len(nodes_ids))]
)

b = fator * matriz_temp

#Calculo do PageRank
m = (1-d)*h + d*b

#Matriz temporaria com uma coluna com valores 1
matriz_temp2 = np.matrix([[1] for i in range(len(nodes_ids))])

#Vetor v normalizado
v = fator * matriz_temp2

### Algoritmo PageRank
Algoritmo utilizado para calcular todas as interações do PageRank. Ele calcua m * v até convergir. A diferença de convergência (soma da diferença de todos os elementos) é definida como 0.001. 

In [142]:
count = 0

def pageRank(v):
    
    global count
    
    count += 1
    
    if sum(abs(m*v-v)) > 0.001:
        return pageRank(m*v)
    else:
        return m*v

### Gerando o resultado do PageRank
Aqui usamos o algoritmo do PageRank para verificar o pageRank final do conjunto de dados e geramos o arquivo csv que será utilizado na ferramenta **Gephi** para gerar o gráfico de visualizações dos resultados. 
<p>
Inicialmente, criamos o dataframe referente aos resultados do PageRank.

In [156]:
resultado = pageRank(v)
resultado = [cell.item(0,0) for cell in resultado]

pagerank_result = pd.DataFrame({'NODE_ID':nodes_ids, 'PAGE_RANK':resultado})

pagerank_result.head()

,NODE_ID,PAGE_RANK
0,1,0.000139
1,2,0.000028
2,4098,0.000003
3,4,0.000037
4,3,0.000004


### Criação do arquivo csv

In [158]:
#pagerank_result.to_csv(path_or_buf='resultado/pagerank_result.csv', index=False)
#dataset.to_csv(path_or_buf='resultado/soc-sign-bitcounotc-filtrado.csv', columns=['SOURCE_ID', 'TARGET_ID'], index=False)